In [ ]:
import getpass
import os

try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    pass

os.environ["LANGSMITH_TRACING"] = "true"
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
        prompt="Enter your LangSmith API key (optional): "
    )
if "LANGSMITH_PROJECT" not in os.environ:
    os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
        prompt='Enter your LangSmith Project Name (default = "default"): '
    )
    if not os.environ.get("LANGSMITH_PROJECT"):
        os.environ["LANGSMITH_PROJECT"] = "default"
if "DEEPSEEK_API_KEY" not in os.environ:
    os.environ["DEEPSEEK_API_KEY"] = getpass.getpass(
        prompt="Enter your DeepSeek API key (required if using DeepSeek): "
    )

In [8]:
import getpass
import os

if not os.environ.get("DEEPSEEK_API_KEY"):
    os.environ["DEEPSEEK_API_KEY"] = getpass.getpass(
        prompt="Enter your DeepSeek API key (required if using DeepSeek): "
    )
    
from langchain_deepseek import ChatDeepSeek

llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

from langchain_core.messages import SystemMessage, HumanMessage

messages = [
    SystemMessage("Translate the following English into Chinese"),
    HumanMessage("Translate the following English into Chinese!"),
]

ai_msg = llm.invoke(messages)

ai_msg.content

'请提供需要翻译的英文内容，我会尽力为您准确翻译成中文。'

In [10]:
ai_msg = llm.invoke("Hello")
ai_msg.content

'Hello! 😊 How can I assist you today?'

In [11]:
for token in llm.stream("Hello"):
    print(token.content, end="|")

|Hello|!| 😊| How| can| I| assist| you| today|?||

In [13]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

prompt = prompt_template.invoke({
    "language": "Chinese",
    "text": "I have a dream"
})

prompt.messages

[SystemMessage(content='Translate the following from English into Chinese', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I have a dream', additional_kwargs={}, response_metadata={})]

In [14]:
prompt.to_messages()

[SystemMessage(content='Translate the following from English into Chinese', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I have a dream', additional_kwargs={}, response_metadata={})]

In [15]:
response = llm.invoke(prompt)
response.content

'我有一个梦想'

In [27]:
from langchain_core.tools import tool

@tool
def add(x: int, y: int) -> int:
    """Add two numbers together."""
    return x + y

@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b

llm_with_tools = llm.bind_tools([add, multiply])

query = "What is 3 * 12? Also, what is 11 + 49?"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.tool_calls)

messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    
llm_with_tools.invoke(messages)


[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_0_71cba42c-6515-448f-9095-e9eea75ae4e3', 'type': 'tool_call'}, {'name': 'add', 'args': {'x': 11, 'y': 49}, 'id': 'call_1_e8765153-97b7-4d94-859e-d04b93b5f650', 'type': 'tool_call'}]


AIMessage(content='- 3 * 12 = 36  \n- 11 + 49 = 60  ', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 285, 'total_tokens': 305, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 256}, 'prompt_cache_hit_tokens': 256, 'prompt_cache_miss_tokens': 29}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_3d5141a69a_prod0225', 'id': 'b0459440-c07c-43f0-a165-dea93d750fe7', 'finish_reason': 'stop', 'logprobs': None}, id='run-291a8ded-6752-4a86-972c-2ef9fd6bf61d-0', usage_metadata={'input_tokens': 285, 'output_tokens': 20, 'total_tokens': 305, 'input_token_details': {'cache_read': 256}, 'output_token_details': {}})